In [8]:
#spark.conf.set('spark.sql.adaptive.coalescePartitions.enabled', 'true')
spark.conf.set('spark.sql.adaptive.enabled', 'false')
spark.conf.set('spark.sql.shuffle.partitions', 20)

In [4]:
yellow_taxi_df = spark.read.option('header', 'true').option('inferSchema', 'true').csv('../data/YellowTaxis_202210.csv')

In [6]:
yellow_taxi_df.rdd.getNumPartitions()


16

In [9]:
def get_data_frameStats(df, column_name):
    output_df = (
        df.withColumn('partition_number', F.spark_partition_id())
        .groupBy('partition_number')
        .agg(
            F.count('*').alias('record count'),
            F.min(column_name).alias('min_column_value'),
            F.max(column_name).alias('max_column_value')

        )
        .orderBy('partition_number')
    )
    return output_df

In [10]:
yellow_taxi_df_grouped = yellow_taxi_df.groupBy('VendorId', 'payment_type').agg(F.sum('total_amount'))

In [13]:
yellow_taxi_df_grouped.write.csv('../data/test_part_nons')

In [11]:
yellow_taxi_df_grouped.show()

+--------+------------+-------------------+
|VendorId|payment_type|  sum(total_amount)|
+--------+------------+-------------------+
|       1|           2|  3407067.989999411|
|       2|           4| 1103.9500000000007|
|       1|           0|  704915.6500000019|
|       1|           4|  73231.74000000034|
|       1|           1|1.790332764000427E7|
|       6|           0|  279048.8899999997|
|       2|           2|  9225232.369998984|
|       1|           3| 186607.23999999848|
|       2|           0| 2899635.7899999204|
|       2|           1|4.708866527998194E7|
|       2|           3| 29.719999999999956|
+--------+------------+-------------------+



In [14]:
get_data_frameStats(yellow_taxi_df_grouped, 'VendorId').show()

+----------------+------------+----------------+----------------+
|partition_number|record count|min_column_value|max_column_value|
+----------------+------------+----------------+----------------+
|               1|           2|               1|               2|
|               4|           1|               1|               1|
|               5|           1|               1|               1|
|               7|           1|               1|               1|
|               8|           1|               6|               6|
|               9|           3|               1|               2|
|              17|           1|               2|               2|
|              18|           1|               2|               2|
+----------------+------------+----------------+----------------+

